In [1]:
import laps
import redis

#### Setup redis client

Bash command to find IP of a container:<br>
`docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' container_name_or_id`<br>
Redis container is (as of now) called redis1

In [ ]:
redis_client = redis.Redis(host="172.17.0.2", port=6379)

Define function to get neighbour nodes.

Currently it will only return neighbours across and not diagonal

In [ ]:
def get_neighbours(point,grid, width, height):
    x,y = point.point
    liste = []
    if x-1 > 0:
        liste.append((x-1, y))
    if x+1 < width:
        liste.append((x+1, y))
    if y-1 > 0:
        liste.append((x, y-1))
    if y+1 < height:
        liste.append((x, y+1))
        
    return liste

In [ ]:
def djikstra(start, goal, grid, width, height):
    start.dist = 0

    openset = set()

    visited = {}
    current = start
    while True:
        if current == goal:
            path = []
            while current.parent:
                path.append(current)
                current = current.parent

            return path

        for n in get_neighbours(current, grid, width, height):
            if (not visited.__contains__(n)):
                openset.add(n)
                if n.dist > current.dist + n.move_cost(current):
                    n.dist = current.dist + n.move_cost(current)
                    n.parent = current

        visited[current] = current
        current = min(openset, key=lambda n: n.dist)
        openset.remove(current)

Setup pathfinding module

In [ ]:
def handler(runner, job):
    print("Hello, World!")
    #runner.send_error("lmao this is an error")
    points = [{"x": 20, "y": 10}, {"x": 10, "y": 20}]

    return (False, runner.response(points))


with laps.Runner("dijkstra_module", "1.0.0", redis_client) as runner:
    runner.run(handler)